# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [3]:
$:.unshift "."
require 'db_setup'
require 'db_load'

false

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [7]:
book = Book.first
puts book.title
puts book.author

Végjáték (regény)
#<Author:0x000000037f2440>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [8]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"

Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [12]:
author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 40, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [13]:
IRuby.html "<h1>Test HTML-a</h1>"

"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [14]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [15]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [24]:
authors = Author.where('born < ?',Date.parse("1900-1-1"))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end

Juliusz Słowacki 04-09-1809
Henryk Sienkiewicz 05-05-1846
Eliza Orzeszkowa 06-06-1841
Adam Mickiewicz 24-12-1798
Jack London 12-01-1876
Mikołaj Kopernik 19-02-1473
Maria Franciszka Kozłowska 27-05-1862
William Szekspir 23-04-1564
John Ronald Reuel Tolkien 03-01-1892
Aleksander Dumas (ojciec) 24-07-1802
Tadeusz Dołęga-Mostowicz 10-08-1898
Vladimir Nabokov 22-04-1899
Adolf Hitler 20-04-1889
Benedykt Chmielowski 20-03-1700
Aldous Huxley 26-07-1894
Honoré de Balzac 20-05-1799
Franz Kafka 03-07-1883
Howard Phillips Lovecraft 20-08-1890
Lew Tołstoj 09-09-1828
Ilja Ilf 15-10-1897
Gabriela Zapolska 30-03-1857
Clive Staples Lewis 29-11-1898
Iwan Turgieniew 09-11-1818
Lucy Maud Montgomery 30-11-1874
Aleksander Majkowski 17-07-1876


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-15", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 15, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 18, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 22, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 23, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 25, name: "William", surname: "Szekspir", born: "156

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [26]:
authors = Author.where(died:(Date.parse("1901-1-1")..Date.parse("2000-12-31")))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")} "
end

Henryk Sienkiewicz 05-05-1846 15-11-1916 
Eliza Orzeszkowa 06-06-1841 18-05-1910 
Jack London 12-01-1876 22-11-1916 
Antoni Gołubiew 25-02-1907 27-06-1979 
George Orwell 25-06-1903 21-01-1950 
John Ronald Reuel Tolkien 03-01-1892 02-09-1973 
Tadeusz Dołęga-Mostowicz 10-08-1898 20-09-1939 
Vladimir Nabokov 22-04-1899 02-07-1977 
Adolf Hitler 20-04-1889 30-04-1945 
Aldous Huxley 26-07-1894 22-11-1963 
Franz Kafka 03-07-1883 03-06-1924 
Howard Phillips Lovecraft 20-08-1890 15-03-1937 
Karol Olgierd Borchardt 25-03-1905 20-05-1986 
Lew Tołstoj 09-09-1828 20-11-1910 
Janusz Zajdel 15-08-1938 19-07-1985 
Isaac Asimov 04-10-1919 06-04-1992 
Ilja Ilf 15-10-1897 13-04-1937 
Jewgienij Pietrow (pisarz) 13-12-1903 02-07-1942 
Gabriela Zapolska 30-03-1857 17-12-1921 
Clive Staples Lewis 29-11-1898 22-11-1963 
Lucy Maud Montgomery 30-11-1874 24-04-1942 
Alfred Szklarski 21-01-1912 09-04-1992 
Aleksander Majkowski 17-07-1876 10-02-1938 
Ian Fleming 28-05-1908 12-08-1964 


[#<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-15", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-18", image_url: nil>, #<Author id: 18, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 21, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 24, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 27, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 29, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [35]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><h3> #{author.born}</h3><img src='#{author.image_url}'/>"
end

authors = Author.where("image_url is not null").order(born: :desc).limit(5)
document = ''
authors.each do |author|
  document += display_author(author)
end
IRuby.html document

"<h1>Andrzej Pilipiuk</h1><h3> 1974-03-20</h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Andrzej_Pilipiuk-Polcon2006.jpg?width=300'/><h1>J. K. Rowling</h1><h3> 1965-07-31</h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1>Rafał A. Ziemkiewicz</h1><h3> 1964-09-13</h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rafał_Ziemkiewicz.jpg?width=300'/><h1>Boris Akunin</h1><h3> 1956-05-20</h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Boris_Akunin_2012.jpg?width=300'/><h1>Orson Scott Card</h1><h3> 1951-08-24</h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [40]:
def display_book(book)
  "<h1>#{book.title}</h1><h3> #{book.published}</h3><img src='#{book.image_url}'/>"
end

books = Book.where("image_url is not null").order(:published).limit(1)
document = ''
books.each do |book|
  document += display_book(book)
end
IRuby.html document

"<h1>Hamlet</h1><h3> </h3><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [41]:
def display_book(book)
  "<h1>#{book.title}</h1><img src='#{book.image_url}'/>"
end

books = Book.where("published is null").all
document = ''
books.each do |book|
  document += display_book(book)
end
IRuby.html document

"<h1>Végjáték (regény)</h1><img src=''/><h1>Hamlet</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/>"

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.